# Pre-requisits
- "Models training helpers.ipynb should be contained in the same directory as this file
    in order to run command %run "Models training helpers.ipynb" sort=False and utilize the helpers
- Glove should be downloaded to 'Data/Iterative-models-building/Training data/glove.6B.100d.t
    this is where Models training helpers.ipynb will pick it up   


<h1>Iterative Model Building - Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pre-requisits" data-toc-modified-id="Pre-requisits-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pre-requisits</a></span></li><li><span><a href="#Import-&quot;Models-training-helpers.ipynb-and-3-helper-classes." data-toc-modified-id="Import-&quot;Models-training-helpers.ipynb-and-3-helper-classes.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import "Models training helpers.ipynb and 3 helper classes.</a></span></li><li><span><a href="#Setup-Github-credentials-and-repos-etc-to-be-used-for-reading-data-and-in-some-cases-pushing-data-directly-to-github" data-toc-modified-id="Setup-Github-credentials-and-repos-etc-to-be-used-for-reading-data-and-in-some-cases-pushing-data-directly-to-github-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Setup Github credentials and repos etc to be used for reading data and in some cases pushing data directly to github</a></span></li><li><span><a href="#Definition-of-the-various-paths-both-locally-and-directly-in-github" data-toc-modified-id="Definition-of-the-various-paths-both-locally-and-directly-in-github-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Definition of the various paths both locally and directly in github</a></span></li><li><span><a href="#Data-gathering" data-toc-modified-id="Data-gathering-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data gathering</a></span></li><li><span><a href="#Clasification---load-the-classifiers-from-pickle-files" data-toc-modified-id="Clasification---load-the-classifiers-from-pickle-files-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Clasification - load the classifiers from pickle files</a></span></li><li><span><a href="#Pass-all-the-combined-convention-sentences-through-each-convention-classifier." data-toc-modified-id="Pass-all-the-combined-convention-sentences-through-each-convention-classifier.-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Pass all the combined convention sentences through each convention classifier.</a></span></li><li><span><a href="#Annotate-data-for-each-type-of-classifications-(high-confidence,-medium,-low-+-the-confidence-score)" data-toc-modified-id="Annotate-data-for-each-type-of-classifications-(high-confidence,-medium,-low-+-the-confidence-score)-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Annotate data for each type of classifications (high confidence, medium, low + the confidence score)</a></span></li><li><span><a href="#Stort-Conventions-classification-results-at:" data-toc-modified-id="Stort-Conventions-classification-results-at:-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Stort Conventions classification results at:</a></span></li><li><span><a href="#Next-steps-to-be-done-manually-via-the-audit-tool,-which-will-created-audit-files-at-location-AUDITED_DATA_DIR_GIT" data-toc-modified-id="Next-steps-to-be-done-manually-via-the-audit-tool,-which-will-created-audit-files-at-location-AUDITED_DATA_DIR_GIT-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Next steps to be done manually via the audit tool, which will created audit files at location AUDITED_DATA_DIR_GIT</a></span></li><li><span><a href="#Collect-Audited-Data-for-purpose-of-retraining." data-toc-modified-id="Collect-Audited-Data-for-purpose-of-retraining.-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Collect Audited Data for purpose of retraining.</a></span></li><li><span><a href="#Retrain-the-models-with-the-new-audited-data-and-training-data" data-toc-modified-id="Retrain-the-models-with-the-new-audited-data-and-training-data-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Retrain the models with the new audited data and training data</a></span></li><li><span><a href="#Get-Stopwords" data-toc-modified-id="Get-Stopwords-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Get Stopwords</a></span></li></ul></div>

# Import "Models training helpers.ipynb and 3 helper classes. 


In [1]:
%run "All_helper_classes.ipynb" sort=False

#from "Models training helpers.ipynb" we will import 3 helper classes and the associated helper methods.
parsing_helpers = generic_parsing_helpers()
aivm_helper = AIVM_helper()
model_helpers = default_model_helpers_for_project()


Using TensorFlow backend.


Necessary libraries imported.
Initialised generic_parsing_helpers class and methods
Initialised AIVM_helper class and methods
Initialised model_helpers class and methods


In [2]:
%%javascript
//To remove scroll from any output area and automatically extend the jupyter cell
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Setup Github credentials and repos etc to be used for reading data and in some cases pushing data directly to github

*****Rember to add your github user and passeord here*******

In [3]:
#To read all the data files directly from  github
USE_GIT_HUB = True 
READ_PICKLE_FROM_CLOUD = False #we have no central location as of yet for pickel files

###*******GIT Hub repo ********###
GIT_REPO = 'AIVC'
GIT_OWNER = 'aideenf'
GIT_OWNER_REPO = GIT_OWNER + '/'+ GIT_REPO

#########To push data files directly to  github - usr/pssword required######
GIT_USER = 'aideenf' 
GIT_PSWD = 'aid99rk4'
###*******END-POPULATE GIT_HUB Credentials ********#########################

# Definition of the various paths both locally and directly in github

In [4]:
#The paths are now defined in a helper class, in order to use paths.DIR
paths = project_paths()

# Data gathering

To be done for different sources, to have high variability. It's important to keep tracking of where does each sentence come from (add a label of provenance).
Identified data sources:
    - Google
    - Github
    - Semantic Scholar
    - pdf files

In [5]:
def aggrigate_gathered_data_from_git (gathered_data_dir_git,
                            github_gathered_url, 
                            git_owner,
                            git_repo) :

    gathered_data_dir_git = gathered_data_dir_git
    github_gathered_url_path = github_gathered_url

    git_owner = git_owner
    git_repo = git_repo
    
    gathered_conventions_files = []
    gathered_conventions_data = {}
    
    gathered_conventions_files = aivm_helper.list_files_from_github_dir (git_owner, git_repo, gathered_data_dir_git)
    gathered_conventions_files = aivm_helper.remove_excluded_files(gathered_conventions_files)
                   #aivm_helper.remove_excluded_files_except(my_file_list, "audited_training_data")
    
    display (gathered_conventions_files)
    #Read conventions data file from github. 
    for f in gathered_conventions_files:
         gathered_conventions_data[aivm_helper.clean_file_name(f)] = pd.read_csv(github_gathered_url_path+f, sep='\t', error_bad_lines=False)
        # NOTE: "truncated": false  we should check for truncated = true to do follow on call to get all files

    
    return gathered_conventions_files, gathered_conventions_data
    gc.collect()
    
   

In [6]:
gathered_conventions_files = []
gathered_conventions_data = {}

if(USE_GIT_HUB == False):   
    #For each file() in gathered conventions folder
    gathered_conventions_files = [f for f in os.listdir(paths.GATHERED_DATA_CONV_DIR_LOCAL)]
    gathered_conventions_files = aivm_helper.remove_excluded_files(gathered_conventions_files) 
    for f in gathered_conventions_files:
        gathered_conventions_data[aivm_helper.clean_file_name(f)] = pd.read_csv(os.path.join(paths.GATHERED_DATA_CONV_DIR_LOCAL, f), sep='\t')
        

if(USE_GIT_HUB == True): 
        
    gathered_conventions_files, gathered_conventions_data = aggrigate_gathered_data_from_git (paths.GATHERED_DATA_CONV_DIR_GIT,
                            paths.GITHUB_GATHERED_URL_PATH, 
                            GIT_OWNER,
                            GIT_REPO)

        

['gathered_github_sentences.tsv',
 'gathered_green_test.tsv',
 'gathered_news_sentences.tsv',
 'gathered_pdf_green_AIFORSUSTAINABLEINTELIGENTBUILDINGS.tsv',
 'gathered_pdf_green_Harnessing_Artificial_Intelligence_for_the_Earth_report_2018.tsv',
 'gathered_pdf_green_introduction_to_the_green_economy_approach.tsv',
 'gathered_s2_17-19_ki_kw.tsv']

In [7]:
display ("Now we have a dictionary(gathered_conventions_data) of dataframes one entry for each gathered sentence source:" , gathered_conventions_files)

display (gathered_conventions_data[list(gathered_conventions_data.keys())[0]].head(1))

'Now we have a dictionary(gathered_conventions_data) of dataframes one entry for each gathered sentence source:'

['gathered_github_sentences.tsv',
 'gathered_green_test.tsv',
 'gathered_news_sentences.tsv',
 'gathered_pdf_green_AIFORSUSTAINABLEINTELIGENTBUILDINGS.tsv',
 'gathered_pdf_green_Harnessing_Artificial_Intelligence_for_the_Earth_report_2018.tsv',
 'gathered_pdf_green_introduction_to_the_green_economy_approach.tsv',
 'gathered_s2_17-19_ki_kw.tsv']

,sentence,provenance
0,react can also render on the server using node...,Github


# Clasification - load the classifiers from pickle files

Classify, using each of the classifiers, the gathered sentences
Right now the pickle files are always read from the local machine as they are not compressed and are too big to store on gitHub. it will be convenient if we either
    - push to another centralized location automatically on creation
    
in order to avoid the need for manual synchronization. Note that the code to read from cloud is partially implemented. see below! 

In [8]:
## Read pickled classifiers
# Note: pickle is used for serializing and de-serializing a Python object structure. 
# Any object in python can be pickled so that it can be saved on disk. 
# What pickle does is that it “serialises” the object first before writing it to file. 
# Pickling is a way to convert a python object (list, dict, etc.) into a character stream

## read convention models from pickle file from gitHub or local 
if(READ_PICKLE_FROM_CLOUD == True): 
     
    #make sure updated pickle files are in github
    convention_convnet_items = pickle.load(urllib.request.urlopen(paths.CONV_MODEL_PCKL_CLOUD_URL))
    characteristics_convnet_items = pickle.load(urllib.request.urlopen(paths.CHARACT_MODEL_PCKL_CLOUD_URL))
    
    
elif(READ_PICKLE_FROM_CLOUD == False):
    ## Load the  convention models from the downloaded pickle file             
    _conventions_models, _conventions_tokenizers, _conventions_data_val_x ,_conventions_data_val_y ,_conventions_train_histories = model_helpers.read_DL_models_from_pickle( 'conv_models_items.pickle', paths.MODELS_DIR_LOCAL)

In [9]:
data_sentences = []
data_sentences_provenance = []

#for each data source in the dataframe dictionary
for k in gathered_conventions_data.keys():
    #retrieve the dataFrame for the convention
    conv_data_df = gathered_conventions_data[k]
    #aggregate the source
    provenances = conv_data_df['provenance'].values
    #aggregate the sentence
    texts = conv_data_df['sentence'].values
    for i in range(len(conv_data_df)):
        data_sentences.append( texts[i])
        data_sentences_provenance.append(provenances[i])

#Now we have a list of sentences and sentence sources which we will 
#input to the trained model and later we will evaluate and audit 
display (HTML('Example sentence/source from combination of all data sources:'))
display (HTML('<b>Sentence: </b>'+ data_sentences[0]))
display (HTML('<b>Source: </b>' + data_sentences_provenance[0]))
gc.collect()

8

# Pass all the combined convention sentences through each convention classifier. 

In [10]:
conventions_classifications = model_helpers.calculate_matches_DL( data_sentences, _conventions_models, _conventions_tokenizers)
display (conventions_classifications)
gc.collect()

{'training_Domestic.txt': array([0.49484226, 0.5286302 , 0.3985585 , ..., 0.01692784, 0.01085482,
        0.21079467], dtype=float32),
 'training_Civic.txt': array([0.04898009, 0.00690331, 0.0257176 , ..., 0.00725713, 0.00043306,
        0.01036146], dtype=float32),
 'training_Project.txt': array([8.4338385e-01, 2.3925945e-01, 7.5533372e-01, ..., 1.7265688e-01,
        8.0423959e-04, 9.7571075e-01], dtype=float32),
 'training_Inspired.txt': array([0.00118321, 0.32361186, 0.02998996, ..., 0.02082202, 0.2663042 ,
        0.00473872], dtype=float32),
 'training_Green.txt': array([0.00276766, 0.2844936 , 0.042117  , ..., 0.08290219, 0.01610001,
        0.01512012], dtype=float32),
 'training_Market.txt': array([1.6413704e-01, 1.4360935e-03, 2.6892603e-04, ..., 7.1194448e-04,
        1.2051293e-02, 1.0864363e-05], dtype=float32),
 'training_Industrial.txt': array([0.7737953 , 0.18235698, 0.05561984, ..., 0.37716824, 0.99817514,
        0.9960849 ], dtype=float32),
 'training_Renown.txt': ar

7

# Annotate data for each type of classifications (high confidence, medium, low + the confidence score)

In [11]:
class ClassificationResult:
        
    def __init__(self, text, value, level, provenance='Unknown'):
        self.text = text
        self.confidence_value = value
        self.confidence_level = level
        self.data_provenance = provenance
        
    def __str__(self):
        return "{} -- {} -- {}".format(self.text, self.confidence_value, self.confidence_level)
    
    def __repr__(self):
        return "{} -- {} -- {}".format(self.text, self.confidence_value, self.confidence_level)
        
        
def split_sentences_by_confidence (calculated_classifications, _sentences, _sentences_provenance):

    stratified_classifications = {}

    for k in calculated_classifications.keys():
        classifications = calculated_classifications[k]

        ## Low level percentile
        low_percentile = np.percentile(classifications, 33)#0.2
        classifications_low = np.where(classifications<=low_percentile)[0]
        #print(classifications_low)

        ## Medium level percentile
        medium_percentile = np.percentile(classifications, 66)#0.8
        classifications_medium = np.where((classifications<=medium_percentile) & (classifications>low_percentile))[0]
        #print(classifications_medium)

        ## High level percentile
        top_percentile = np.percentile(classifications, 100) #1.0
        classifications_top = np.where((classifications<=top_percentile) & (classifications>medium_percentile))[0]
        #print(classifications_top)
       
        classified_sentences = []
        #print(classifications_low)
        for i1 in classifications_low:
            c1 = ClassificationResult(_sentences[i1], classifications[i1], "Low", _sentences_provenance[i1])
            classified_sentences.append(c1)

        for i2 in classifications_medium:
            c2 = ClassificationResult(_sentences[i2], classifications[i2], "Medium", _sentences_provenance[i2])
            classified_sentences.append(c2)

        for i3 in classifications_top:
            c3 = ClassificationResult(_sentences[i3], classifications[i3], "High", _sentences_provenance[i3])
            classified_sentences.append(c3)
            
        stratified_classifications[aivm_helper.clean_file_name(k)] = classified_sentences
    return stratified_classifications

# Stort Conventions classification results at:
paths.CLASSIFIED_DATA_DIR_LOCAL 
paths.CLASSIFIED_DATA_DIR_GIT directly to git if this option is activated.

in format "training_{convention}_stratified_classifications.tsv"

###Question: why is the naming convention "training" when these have to be audited first. Can we simply name them
"{convention}_stratified_classifications.tsv"


In [12]:
print("paths.CLASSIFIED_DATA_DIR_LOCAL:", paths.CLASSIFIED_DATA_DIR_LOCAL)
print("paths.CLASSIFIED_DATA_DIR_GIT:", paths.CLASSIFIED_DATA_DIR_GIT)

paths.CLASSIFIED_DATA_DIR_LOCAL: ./Data/Iterative-models-building/Classification results/Conventions/
paths.CLASSIFIED_DATA_DIR_GIT: cp_wssc/Data/Iterative-models-building/Classification results/Conventions/


In [13]:
conv_stratified_classifications = split_sentences_by_confidence(conventions_classifications, data_sentences, data_sentences_provenance)

file_list = []
file_list_git = []
  
for k in conv_stratified_classifications.keys():
    with open(os.path.join(paths.CLASSIFIED_DATA_DIR_LOCAL, "{}_stratified_classifications.tsv".format(k)), "w")as f3:
        
        file_list.append(paths.CLASSIFIED_DATA_DIR_LOCAL + "{}_stratified_classifications.tsv".format(k))
        file_list_git.append(paths.CLASSIFIED_DATA_DIR_GIT + "{}_stratified_classifications.tsv".format(k))
        
        f3.write("{}\t{}\t{}\t{}\t{}\n".format("text", "convention", "confidence_value", "confidence_level", "data_provenance"))
    
        for c in conv_stratified_classifications[k]:
            convention = k.replace("training_", "").lower()     
            f3.write("{}\t{}\t{}\t{}\t{}\n".format(c.text, convention, c.confidence_value, c.confidence_level, c.data_provenance))
        f3.close()
        
gc.collect()
if(USE_GIT_HUB == True):
#push to github
        my_file_list = file_list
        push_to_git_as = file_list_git
        commit, message = aivm_helper.save_to_github(GIT_USER, GIT_PSWD, GIT_OWNER_REPO, my_file_list, push_to_git_as, "auto push classified conventions")

        if (commit != "error"):
            print ("File to commit:", my_file_list)
            print ("Push to git as:", push_to_git_as)
            print ("Commit: ", commit)
            display(HTML("<font color='green'><b>Files Merged and Saved!!</b></font>"))
        
            
        if (commit == "error"):
            print ("File to commit: ", my_file_list)
            print ("Push to git as: ", push_to_git_as)
            display(HTML("<font color='red'><b>Warning!!</b></font>"))
            print(message)

In [ ]:
##############CODE ADDED BY AIDEEN#######################################################
#Create an aggregated DataFrame and then a tsv/csv file of the full data for auditing
#This code is to create a file and push it to git hub to be read by the audit tool only
# This is independant piece of code, removing it will only remove this functionality
# leaving it here means that when we retrain the model this file will be auto updated with
# new labels. (to discuss with David - to revert this to the format that can be used by the 
# model training process)
df_list = [pd.read_csv(file, sep='\t') for file in file_list]
#concatenate them 
print("Concatenate all files to one")
convention_sentences_df = pd.concat(df_list)
## We will perform one hot encoding
print("Applying one hot encoding to convention to have format suitable for audit tool (binary and for all)")
convention_sentences_df["convention"] = pd.Categorical(convention_sentences_df["convention"])
dfDummies = pd.get_dummies(convention_sentences_df["convention"], prefix="convention")
convention_sentences_df = pd.concat([convention_sentences_df, dfDummies], axis=1)
convention_sentences_df = convention_sentences_df.drop(["convention"], axis=1)

## Now we will save a file 'audit_training_data.tsv'  and zipped version locally which contains all of the results
file_name = 'audit_training_data.tsv'
file_name_zip = 'audit_training_data.gz'

convention_sentences_df.to_csv(paths.CLASSIFIED_DATA_DIR_LOCAL + file_name)
convention_sentences_df.to_csv(paths.CLASSIFIED_DATA_DIR_LOCAL + file_name_zip, compression='gzip')

## We will also push this to git hub for use by audit tool
if(USE_GIT_HUB == True):
    #Call helper file to zip the file for push to github
    my_file_list = [paths.CLASSIFIED_DATA_DIR_LOCAL + file_name_zip]
    push_to_git_as = [paths.CLASSIFIED_DATA_DIR_GIT +file_name_zip]

    commit, message = aivm_helper.save_to_github(GIT_USER, GIT_PSWD, GIT_OWNER_REPO, my_file_list, push_to_git_as, "auto push for audit")

    if (commit != "error"):
        print ("File to commit:", my_file_list[0])
        print ("Push to git as:", push_to_git_as[0])
        print ("Commit: ", commit)
        display(HTML("<font color='green'><b>Files Merged and Saved!!</b></font>"))
        
            
    if (commit == "error"):
        print ("File to commit: ", my_file_list[0])
        print ("Push to git as: ", push_to_git_as[0])
        display(HTML("<font color='red'><b>Warning!!</b></font>"))
        print(message)

display (convention_sentences_df.head(2))
display (convention_sentences_df.tail(2))

############################END CODE ADDED BY AIDEEN####################################################

In [ ]:
print(conv_stratified_classifications['training_Domestic'][0].text)
print(conv_stratified_classifications['training_Domestic'][1].text)

# Next steps to be done manually via the audit tool, which will created audit files at location AUDITED_DATA_DIR_GIT



Audit vai tool

filter via provrnance or convention

# Collect Audited Data for purpose of retraining. 
The files that have been produced by the audit tool are retrieved and consolodated into a data frame, this data frame is then converted to tsv file to produce one TSV file with all audited results.

This file will be called... 'audited_ALL_{date}-{time}.tsv' 

Any other process that wishes to use this file should look for the most recent 'audited_ALL_{date}-{time}.tsv'  or retrieve new version with this method

In [33]:
#Method to aggregate the audited data and to return a pandas DB containing the aggregated content. 
def aggrigate_audited_data_from_git (audited_data_dir_git,
                            github_audit_url, 
                            audited_data_dir_local,
                            git_user, 
                            git_pswd, 
                            git_owner_repo,
                            git_owner,
                            git_repo) :

    audited_data_dir_git = audited_data_dir_git
    github_audit_url = github_audit_url
    audited_data_dir_local = audited_data_dir_local
    git_user = git_user
    git_pswd = git_pswd
    git_owner_repo = git_owner_repo
    git_owner = git_owner
    git_repo = git_repo

    # list all files from 'Audited' folder at cp_wssc/Data/Iterative-models-building/Classification%20results/Conventions/Audited/
    # Helper file call to Returns a list of files from the github owner, repo and dir_ref
    my_file_list = aivm_helper.list_files_from_github_dir (git_owner, git_repo, audited_data_dir_git)
    ## Helper file call to Remove our standard excluded files and all files that are not "audited_training_data"
    my_file_list = aivm_helper.remove_excluded_files_except(my_file_list, "audited_training_data")
    if (len(my_file_list) > 0):
        display (my_file_list.sort())
        #Returns a dataframe of concatinated files from github, of names from file_list from the github url specified.
        audited_df = aivm_helper.concat_files_from_github_dir (github_audit_url, my_file_list)
        date, time = aivm_helper.time_stamp()
        # file_name = 'audited_ALL_' + date + '-' + time + '.gz' # If using compression
        #df.to_csv(FILE_PATH + file_name, sep='\t', compression='gzip') # If using compression

        file_name = 'audited_ALL_' + date + '-' + time + '.tsv'
        file_name_unique = 'audited_ALL.tsv'
        
        audited_df.to_csv(audited_data_dir_local + file_name, sep='\t')

        #push to github
        my_file_list = [audited_data_dir_local + file_name, audited_data_dir_local + file_name]
        push_to_git_as = [audited_data_dir_git + file_name_unique, audited_data_dir_git + file_name]

        commit, message = aivm_helper.save_to_github(git_user, git_pswd, git_owner_repo, my_file_list, push_to_git_as, "auto push aggregated audit")

        if (commit != "error"):
            print ("File to commit:", my_file_list[0])
            print ("Push to git as:", push_to_git_as[0])
            print ("Commit: ", commit)
            display(HTML("<font color='green'><b>Files Merged and Saved!!</b></font>"))
        
            
        if (commit == "error"):
            print ("File to commit: ", my_file_list[0])
            print ("Push to git as: ", push_to_git_as[0])
            display(HTML("<font color='red'><b>Warning!!</b></font>"))
            print(message)
    
        return audited_df, message
    else:
        print ("*****NO FILES RETRIEVED******")
    gc.collect()
    
    


In [ ]:
#Call the aggrigate_audited_data method to have the aggrigated data regenerated and a pandas data frame returned.
audited_df, message = aggrigate_audited_data_from_git (paths.AUDITED_DATA_DIR_GIT,
                            paths.GITHUB_AUDIT_URL_PATH, 
                            paths.AUDITED_DATA_DIR_LOCAL,
                            GIT_USER, 
                            GIT_PSWD, 
                            GIT_OWNER_REPO,
                            GIT_OWNER,
                            GIT_REPO)

display (audited_df.head(5))
print (message)
print ("The number of audited samples is", audited_df.shape[0])

In [ ]:
#Method to retrieve a pandas df created by a direct call to the audited data in github. 
def retrieve_aggrigate_audited_data_from_git (path) :
    print (path)
    df = pd.read_csv(path, sep='\t', error_bad_lines=False)
    gc.collect()
    return df
    

In [ ]:
# calling the last created aggregation of the audited data.
#audited_df = retrieve_aggrigate_audited_data_from_git (GITHUB_AGGREGATED_AUDIT_URL_FILE) 


# Retrain the models with the new audited data and training data - still to be reviewed

In [ ]:
mypath = "Data/Training data/Conventions/"
training_files = [f for f in os.listdir(mypath) if (os.path.isfile(os.path.join(mypath, f)) and "training" in f and not f.startswith( '.' )) and not "random" in f]#[:2]
training_files

In [ ]:
FEATURES_LENGTH = 3000
VALIDATION_SPLIT = 0.25
GENERATE_NEW_TRAINING_FILES = False

# Get Stopwords
We should also read these directly from github

In [ ]:
stopwords= parsing_helpers.get_stop_words(paths.STOP_WORDS_PATH)

In [ ]:
_conv_ML_models = {}
_test_data_x = {}
_test_data_y = {}


for f in training_files:
    print("----------------------------------------------------------------")
    print("            {}                  ".format(f))
    print("----------------------------------------------------------------")

    
    
    ## USING licensing text from github
    data_train = pd.read_csv(mypath+f, sep='\t')

    #data_train = pd.read_csv('Data/TRAINING_0/training_contributions.txt', sep='\t')

    print(data_train.shape)

    _texts = []
    _labels = []

    for idx in range(data_train.text.shape[0]):
        
        text = [data_train.text[idx]] #BeautifulSoup(data_train.text[idx])
        
        _texts.append(clean_str(text))#.encode('ascii','ignore')))
        _labels.append(int(data_train.category[idx]))
        
    #SHUFFLE and DATA SPLITTING 
    x_train, x_val, y_train, y_val = train_test_split(_texts, _labels, test_size=VALIDATION_SPLIT, random_state=42)
    
    _test_data_x[f] = np.array(x_val)[np.where(np.array(y_val)==1)]
    _test_data_y[f] = np.array(y_val)[np.where(np.array(y_val)==1)]
    
    model1 = train_new_text_pipelineNB(x_train, y_train)
    tmp_pred = model1.predict(x_val)
    print("Accuracy: {}".format(np.mean(tmp_pred == y_val)))

    _conv_ML_models[f] = model1
    
    #print("  Naive-Bayes: ",mean1)
    #print("  Random-Forest: ",mean2)
    #print(metrics.classification_report(_labels, pred))
        
    print("\n\n\n")


In [ ]:
#conv_ML_matches = calculate_matches_ML( data_sentences, _conv_ML_models)
conv_ML_matches = calculate_matches_mixture( data_sentences, _conv_ML_models, _conventions_models, _conventions_tokenizers)

In [ ]:
_conv_ML_models['training_Domestic.txt'].predict_proba([data_sentences[0]])

In [ ]:
def get_confidence_value(C):
    return C.confidence_value

In [ ]:
## Conventions results sampling
conv_stratified_classifications = split_sentences_by_confidence(conv_ML_matches, data_sentences, data_sentences_provenance)

for k in conv_stratified_classifications.keys():
    with open(os.path.join(BASE_DIR, "Classification results", CONVS_DIR_NAME, "ML_{}_stratified_classifications.tsv".format(k)), "w")as f3:

        f3.write("{}\t{}\t{}\t{}\n".format("text", "confidence_value", "confidence_level", "data_provenance"))
    
        for c in sorted(conv_stratified_classifications[k],key=get_confidence_value, reverse=True):
            
            f3.write("{}\t{}\t{}\t{}\n".format(c.text, c.confidence_value, c.confidence_level, c.data_provenance))

        f3.close()
    

In [ ]:
conv_stratified_classifications.keys()

In [ ]:
conv_stratified_classifications['training_Civic'],